# Lec 16 Lab: Ridge Regression
## CMSE 381 - Fall 2022
## Oct 17, 2022



In this module we are going to test out the ridge regression method we discussed in class from Chapter 6.2.

In [ ]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time


# ML imports we've used previously
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Loading in the data

Ok, here we go, let's play with a baseball data set. 

In [ ]:
hitters_df = pd.read_csv('Hitters.csv')
hitters_df.head()

Annoyingly enough we have some missing values in the data. 

In [ ]:
print("Number of null values:", hitters_df["Salary"].isnull().sum())


So let's go clean those up.....

In [ ]:
# Print the dimensions of the original Hitters data (322 rows x 20 columns)
print("Dimensions of original data:", hitters_df.shape)

# Drop any rows the contain missing values, along with the player names
hitters_df = hitters_df.dropna().drop('Player', axis=1)

# Print the dimensions of the modified Hitters data (263 rows x 20 columns)
print("Dimensions of modified data:", hitters_df.shape)

# One last check: should return 0
print("Number of null values:", hitters_df["Salary"].isnull().sum())

hitters_df.head()

And finally, we can replace our categorical variables with dummy variables.

In [ ]:
hitters_df = pd.get_dummies(hitters_df, drop_first = True)
hitters_df.head()

In [ ]:
y = hitters_df.Salary

# Drop the column with the independent variable (Salary)
X = hitters_df.drop(['Salary'], axis = 1).astype('float64')

X.info()

# Ridge Regression

In class, we learned that doing ridge regression means that we try to find the best model accoding to the score
$$
RSS + \lambda \sum_{i} \beta_i^2.
$$
The good news is that `scikitlearn` has a built in `Ridge` function.  

- [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
- [User guide](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [ ]:
from sklearn.linear_model import Ridge

The bad (ok, not honestly that bad) news is that they call their $\lambda$ parameter $\alpha$. So we're just going to minimize 
$$
RSS + \alpha \sum_{i} \beta_i^2.
$$
instead. So if I pick an alpha value, I can do ridge regression as follows.

In [ ]:
ridge = Ridge(normalize = True) 

a = 1 #<------ this is me picking an alpha value

ridge.set_params(alpha = a)
ridge.fit(X, y)

print('intercept:', ridge.intercept_)
print('\n')
print(pd.Series(ridge.coef_, index = X.columns))
print('\nTraining MSE:',mean_squared_error(y,ridge.predict(X)))
    



&#9989; **<font color=red>Q:</font>** What is the `normalize=True` bit doing in the code above?

* Your answer here*

Of course, that was just me picking a random $\alpha$ out of a hat so there's no reason to trust that it's a good one. I could sit here all day and move that $\alpha$ around to see what's going on, but why do that, when I can make a for loop!

Here's a pile of $\alpha$s for us to test on.

In [ ]:
alphas = 10**np.linspace(4,-2,100)*0.5
alphas = np.append(alphas,0)
alphas

First off, let's take a look at how the coefficients learned change for various choices of $\alpha$. 

Associated with each alpha value is a vector of ridge regression coefficients, which we'll store in a matrix coefs. In this case, it is a  19×100  matrix, with 19 rows (one for each predictor) and 100 columns (one for each value of alpha). 



In [ ]:
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)
    
np.shape(coefs)

First off, let's take a look at how the coefficients learned change for various choices of $\alpha$. 

Associated with each alpha value is a vector of ridge regression coefficients, which we'll store in a matrix coefs. In this case, it is a  19×100  matrix, with 19 rows (one for each predictor) and 100 columns (one for each value of alpha). 

In [ ]:
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')


&#9989; **<font color=red>Q:</font>** There are two variables that have higher magnitude than the rest for low $\alpha$s (read: are either very large positive or very large negative). Which two are they from the data set? Which is which?


In [ ]:
# Your code here #

Now we can start setting up the usual train/test splits to have at least a starting idea of how the testing error is going. The `random_state=1` bit just makes it so that everyone should get the same random split. 

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


&#9989; **<font color=red>Do this:</font>** Train a model using ridge regression with $\alpha = 4$. What is the MSE of your model on the testing set?

In [ ]:
# Your code here #


&#9989; **<font color=red>Do this:</font>** Ha ha nah, you can do better than that.  Lets try all our alphas and take a look at the testing MSE to make a better decision about what $\alpha$ we might want. Modify the code below to plot your testing MSE for all the alphas. What $\alpha$ should we use to train the model?

In [ ]:
# Modify your code from above and add it in the for loop to plot the testing MSE

ridge = Ridge(normalize = True)
errors = []

for a in alphas:
    # ==== Your code goes in here ==== #
    errors.append(17) #<----- random number in here so that the code runs before you fix it
    
np.shape(errors)

plt.plot(alphas,errors)
plt.title('Testing MSE')
ax=plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')



## RidgeCV

Whelp, your meanie professor didn't tell you that there's actually a built in function to do this for you (sorry-not-sorry). Aren't you glad you didn't read ahead?


In [ ]:
from sklearn.linear_model import RidgeCV

- [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)
- [User Guide](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

Basically, `RidgeCV` runs LOOCV (unless you tell it otherwise, see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)) on all the alpha values you specify on an input array, and tells you the best $\alpha$ given that. 

In [ ]:
# I'm going to drop that 0 from the alphas because it makes 
# RidgeCV cranky
alphas = alphas[:-1]


ridgecv = RidgeCV(alphas = alphas, 
                  scoring = 'neg_mean_squared_error', 
                  normalize = True)
ridgecv.fit(X_train, y_train)
print('alpha chosen is', ridgecv.alpha_)

I can predict my values on the test set directly from the `ridgecv` model we just built. 

In [ ]:
pred = ridgecv.predict(X_test)
mean_squared_error(y_test,pred)

This is exactly the same result as if I went and retrained my model using the chosen $\alpha$ using `Ridge`. 

In [ ]:
ridge = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge.fit(X_train, y_train)
mean_squared_error(y_test, ridge.predict(X_test))

&#9989; **<font color=red>Do this:</font>** Why did we get a different best choice of $\alpha$ than we found in the previous section? 

*Your answer here*



-----
### Congratulations, we're done!
Written by Dr. Liz Munch, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.